# 🧠 智能体实践教学：构建你的第一个智能体

## 📘 教学目标
本次Notebook将引导你：
1. 了解什么是智能体（Agent）及其基本结构；
2. 掌握使用 `langchain` 和 `openai` 构建一个具备基础能力的智能体；
3. 亲手实现一个能对话、调用工具并具备记忆功能的智能体。

## 🧩 Step 1：什么是智能体？

我们先来了解一下基础概念。

In [1]:
from IPython.display import Markdown

Markdown('''
**智能体（Agent）是一个能够感知环境、做出判断并执行行动的实体。**
在大模型系统中，它通常具备：
- **语言模型（LLM）**：用于理解指令和生成回应；
- **工具（Tools）**：用于执行搜索、计算、数据库访问等具体任务；
- **记忆（Memory）**：存储上下文对话信息，支持多轮对话；
- **决策逻辑**：决定当前应调用哪个工具或如何响应。
''')


**智能体（Agent）是一个能够感知环境、做出判断并执行行动的实体。**
在大模型系统中，它通常具备：
- **语言模型（LLM）**：用于理解指令和生成回应；
- **工具（Tools）**：用于执行搜索、计算、数据库访问等具体任务；
- **记忆（Memory）**：存储上下文对话信息，支持多轮对话；
- **决策逻辑**：决定当前应调用哪个工具或如何响应。


## 🧪 Step 2：环境配置（必做）

运行以下代码，安装必要库并配置OpenAI的API Key：

In [2]:
%pip install langchain openai faiss-cpu tiktoken langchain-community --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain.chat_models import ChatOpenAI

# 请根据你的OpenAI账号填写以下信息
OPENAI_API_KEY = "sk-b506cb8b46e1464ba64d5de31143be45"
OPENAI_API_BASE = "https://dashscope.aliyuncs.com/compatible-mode/v1"  # 如果你使用的是代理或自定义服务商，可以修改此URL
OPENAI_MODEL = "qwen-plus" 

import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_API_BASE"] = OPENAI_API_BASE

llm = ChatOpenAI(
    model=OPENAI_MODEL,
    temperature=0.9,
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_API_BASE
)

/tmp/ipykernel_273919/2825325141.py:12: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


## 🧱 Step 3：构建你的第一个智能体

### 🛠️ 第一步：定义工具（Tool）

我们将实现一个加法器工具。

In [4]:
from langchain.tools import tool

@tool
def add_numbers(input: str) -> str:
    """接收两个用逗号分隔的数字字符串，如 '3,4' 返回 '7'。支持容错与格式清洗。"""
    try:
        # 移除引号、换行、冒号等常见格式污染
        input = input.strip().replace("'", "").replace("\"", "").replace("\n", "").replace("Observation:", "")
        parts = [x.strip() for x in input.split(',') if x.strip()]
        if len(parts) != 2:
            return "输入格式错误：请提供两个用逗号分隔的数字，如 '3,4'"
        a, b = parts
        return str(int(a) + int(b))
    except Exception as e:
        return f"输入格式错误，请输入形如 '3,4' 的字符串：{str(e)}"

## 阶乘工具！

In [5]:
import re

@tool
def factorial_tool(input_str: str)-> str:
    """接收一个非负整数字符串返回其阶乘值，如输入 '5' 应返回 '120'"""
    try:
        cleaned = re.sub(r"[^\d]", "", input_str.strip())
        if not cleaned:
            return "错误：输入不能为空"
        num = int(cleaned)
        if num < 0:
            return "错误：输入必须是非负整数"

        rst = 1
        for i in range(1, num+1):
            rst *= i
        return str(rst)
    except ValueError:
        return "错误：无效的整数格式"
    except Exception as e:
        return f"未知错误: {str(e)}"

### 🤖 第二步：初始化Agent

In [6]:
from langchain.agents import Tool, initialize_agent
from langchain.agents.agent_types import AgentType

# 添加工具列表
tools = [
    Tool(
        name="AddNumbers",
        func=add_numbers,
        description="用于执行两个数字加法，例如输入 '3,4'"
    ),
    Tool(
        name="Factorial",
        func=factorial_tool,
        description="计算非负整数的阶乘，自动提取输入中数字，如输入5,返回120"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

/tmp/ipykernel_273919/4243128416.py:18: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


### 🧪 第三步：运行测试
langchain中的agent遵循ReAct范式

相关阅读：ReAct: Synergizing Reasoning and Acting in Language Models

In [7]:
agent.run("请帮我计算3加7")

/tmp/ipykernel_273919/339764922.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("请帮我计算3加7")




> Entering new AgentExecutor chain...
我需要使用AddNumbers工具来计算3加7。
Action: AddNumbers
Action Input: 3,7
Observation: 10
Thought:我现在知道最终答案。
Final Answer: 10

> Finished chain.


'10'

In [8]:
agent.run("请帮我计算三和七的和")



> Entering new AgentExecutor chain...
这是一个加法问题，我应该使用AddNumbers工具。
Action: AddNumbers
Action Input: 3,7
Observation: 10
Thought:我现在知道最终答案了。
Final Answer: 三和七的和是10。

> Finished chain.


'三和七的和是10。'

📝 **问题思考**：你能修改输入内容让Agent报错吗？为什么会出错？

🧠 **自主练习**：请你实现一个阶乘计算工具 `factorial_tool`，接收一个正整数字符串返回其阶乘值，如输入 '5' 应返回 '120'。要求：
- 检查输入是否为非负整数
- 对非法输入添加适当错误提示
- 将其添加至 `tools` 列表

## 测试阶乘

In [9]:
agent.run("5的阶乘是多少？") 
agent.run("0的阶乘是多少？") 



> Entering new AgentExecutor chain...
我需要计算5的阶乘。
Action: Factorial
Action Input: 5
Observation: 120
Thought:我现在知道最终答案。  
Final Answer: 5的阶乘是120。

> Finished chain.


> Entering new AgentExecutor chain...
我需要计算0的阶乘。
Action: Factorial
Action Input: 0
Observation: 1
Thought:我现在知道最终答案。  
Final Answer: 0的阶乘是1。

> Finished chain.


'0的阶乘是1。'

In [10]:
agent.run("请先计算3加5，再求结果的阶乘")



> Entering new AgentExecutor chain...
首先需要计算3加5，然后使用得到的和计算阶乘。
Action: AddNumbers
Action Input: 3,5
Observation: 8
Thought:现在我已经得到了3加5的结果为8，接下来需要计算8的阶乘。
Action: Factorial
Action Input: 8
Observation: 40320
Thought:I now know the final answer  
Final Answer: 40320

> Finished chain.


'40320'

## 🔁 Step 4：丰富你的Agent记忆能力

In [11]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

agent_with_memory = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

/tmp/ipykernel_273919/732010614.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")


In [12]:
agent_with_memory.run("请你帮我计算12+47等于多少")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes  
Action: AddNumbers  
Action Input: 12,47  
Observation: 59
Thought:Do I need to use a tool? No  
AI: 12加上47的结果是59。

> Finished chain.


'12加上47的结果是59。'

In [13]:
agent_with_memory.run("我刚才的计算结果是多少？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No  
AI: 您之前的计算结果是59。这是之前计算12加47得到的答案。

> Finished chain.


'您之前的计算结果是59。这是之前计算12加47得到的答案。'

In [14]:
agent_with_memory.run("在刚才的基础上再加23，结果是多少？")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes  
Action: AddNumbers  
Action Input: 59,23  
Observation: 82
Thought:Do I need to use a tool? No  
AI: 在之前计算结果59的基础上再加23，最终结果是82。

> Finished chain.


'在之前计算结果59的基础上再加23，最终结果是82。'

In [15]:
print("\n--- 当前记忆内容 ---")
for msg in memory.chat_memory.messages:
    print(f"{msg.type.upper()}: {msg.content}")


--- 当前记忆内容 ---
HUMAN: 请你帮我计算12+47等于多少
AI: 12加上47的结果是59。
HUMAN: 我刚才的计算结果是多少？
AI: 您之前的计算结果是59。这是之前计算12加47得到的答案。
HUMAN: 在刚才的基础上再加23，结果是多少？
AI: 在之前计算结果59的基础上再加23，最终结果是82。


### 使用摘要记忆
摘要记忆通过将对话历史生成摘要，适合长对话中保持语义一致性而不被信息爆炸干扰。

In [16]:
from langchain.memory import ConversationSummaryMemory

summary_memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history")

agent_summary = initialize_agent(
    tools=tools,
    llm=llm,
    memory=summary_memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True
)

/tmp/ipykernel_273919/902357341.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history")


**练习**：试试连续输入三轮关于旅游计划的问题，观察模型是否能总结用户的目标。

In [17]:
response1 = agent_summary.run("我想春天去北京旅游，有什么建议吗？")
print(f"Response 1: {response1}\n")

response2 = agent_summary.run("我的预算比较有限，有什么节省开支的方法？")
print(f"Response 2: {response2}\n")

response3 = agent_summary.run("我还想去历史名胜古迹，哪里合适？")
print(f"Response 3: {response3}\n")

# 查看记忆摘要
print("当前记忆摘要：")
print(summary_memory.load_memory_variables({})["chat_history"])



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No  
AI: 春天是去北京旅游的绝佳时节，气候宜人，鲜花盛开。以下是一些建议供你参考：

1. **赏花地点**：春天的北京到处都是花开的景象。你可以去玉渊潭公园看樱花，或者去北京植物园观赏桃花和其他花卉。
   
2. **历史景点**：不要错过参观故宫、天安门广场和长城（如八达岭或慕田峪段）。这些地方不仅历史悠久，而且在春天的时候周围环境非常美丽。

3. **文化体验**：可以去看看京剧表演，体验老北京的文化韵味。另外，798艺术区也是一个不错的选择，可以感受现代与传统交融的艺术氛围。

4. **胡同漫步**：走进南锣鼓巷或其他保留完好的胡同区域，体会老北京人的生活方式。

5. **饮食推荐**：尝试一些地道的北京美食，比如烤鸭、炸酱面等。

希望这些建议对你有所帮助！如果你有更多具体需求或问题，随时告诉我。

> Finished chain.
Response 1: 春天是去北京旅游的绝佳时节，气候宜人，鲜花盛开。以下是一些建议供你参考：

1. **赏花地点**：春天的北京到处都是花开的景象。你可以去玉渊潭公园看樱花，或者去北京植物园观赏桃花和其他花卉。
   
2. **历史景点**：不要错过参观故宫、天安门广场和长城（如八达岭或慕田峪段）。这些地方不仅历史悠久，而且在春天的时候周围环境非常美丽。

3. **文化体验**：可以去看看京剧表演，体验老北京的文化韵味。另外，798艺术区也是一个不错的选择，可以感受现代与传统交融的艺术氛围。

4. **胡同漫步**：走进南锣鼓巷或其他保留完好的胡同区域，体会老北京人的生活方式。

5. **饮食推荐**：尝试一些地道的北京美食，比如烤鸭、炸酱面等。

希望这些建议对你有所帮助！如果你有更多具体需求或问题，随时告诉我。



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No  
AI: 在北京旅行时，如果预算有限，您可以采取以下几种节省开支的方法：  

1. **免费景点**：多选择一些无需门票的景点，例如天安门广场、奥林匹克公园、后海等，这些地方同样能

## 🎯 自主练习
请完成以下任务，提交代码与说明：

1. **集成搜索功能**，使用 `DuckDuckGoSearchRun` 工具并问“OpenAI是谁？”；
2. **尝试更换Agent类型**为 `AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION` 并观察变化；

提示：使用如下代码添加搜索工具：
```python
!pip install -U duckduckgo-search
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
tools.append(Tool(name="Search", func=search.run, description="执行网页搜索"))
```


In [18]:
!pip install -U duckduckgo-search

In [19]:
from langchain.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="执行网页搜索,用于回答实时性问题"
    )
]

In [20]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,

    verbose=True
)

In [21]:
response = agent.run("OpenAI是谁？")
print(f"[回答] {response}\n")



> Entering new AgentExecutor chain...
 我应该使用搜索工具来查找有关OpenAI的实时信息。
Action: Search
Action Input: OpenAI是谁？
Observation: ChatGPT的爆炸性成功是OpenAI发展史上最戏剧性的篇章，而这一改变世界的产品，在Sam Altman的描述中，"确实是个意外"。 这个意外不仅彻底改变了OpenAI ... OpenAI的研究成果包括了众多重要的论文和突破性的技术，其中最知名的是GPT系列（Generative Pre-trained Transformer）模型，包括GPT-3。 为了确保 人工智能 的利益能够与整个人类社会的利益保持一致， OpenAI 的使命是确保 人工智能 对于所有人都是有益的， 并 且能够 ... 那么，谁是这个突破性组织背后的推动者？这篇文章将回顾OpenAI 的历史，并介绍那些在其创建过程中发挥重要作用的关键人物。 >>点击领取$100现金大礼! 谁是OpenAI 的创始人？ OpenAI 并没有单一的创始人，而是由多位科技和人工智能领域的重要人物共同成立的。 十年前，OpenAI 诞生在一张厨房餐桌边，创始人们当时想的不是怎么盈利，而是如何让 AI 为人类服务。 十年后，这家已估值千亿、拥有 ChatGPT 的技术巨头，依旧在追问那个最初的问题：AGI 应该属于谁，又该为谁所用？ 因首先发表人工智慧语言大模型而名噪一时的OpenAI CEO山姆．奥特曼（Sam Altman）出席美国国会听证会时表示，美国的AI语言大模型是世界上最先进的 ...
Thought:我找到了关于OpenAI的相关信息。
Final Answer: OpenAI是一家领先的人工智能研究实验室，由多位科技和人工智能领域的重要人物共同创立。它并非由单一创始人创建，而是由一群希望利用AI为人类服务的人士共同成立。OpenAI的研究成果包括众多重要的论文和突破性技术，其中最知名的是GPT系列（Generative Pre-trained Transformer）模型，包括GPT-3。OpenAI的使命是确保人工智能对所有人都是有益的，并且能够与整个人类社会的利益保持一致。Sam Altman是OpenAI的现任CEO，他在推动公司发展方面发挥了重要作用。


In [22]:
memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    return_messages=True
)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)
response = agent.run("OpenAI是谁？")
print(f"[回答] {response}\n")



> Entering new AgentExecutor chain...
```json
{
    "action": "Search",
    "action_input": "OpenAI 是谁"
}
```
Observation: 十年前，OpenAI 诞生在一张厨房餐桌边，创始人们当时想的不是怎么盈利，而是如何让 AI 为人类服务。 十年后，这家已估值千亿、拥有 ChatGPT 的技术巨头，依旧在追问那个最初的问题：AGI 应该属于谁，又该为谁所用？ OpenAI的研究成果包括了众多重要的论文和突破性的技术，其中最知名的是GPT系列（Generative Pre-trained Transformer）模型，包括GPT-3。 为了确保 人工智能 的利益能够与整个人类社会的利益保持一致， OpenAI 的使命是确保 人工智能 对于所有人都是有益的， 并 且能够 ... 那么，谁是这个突破性组织背后的推动者？这篇文章将回顾OpenAI 的历史，并介绍那些在其创建过程中发挥重要作用的关键人物。 >>点击领取$100现金大礼! 谁是OpenAI 的创始人？ OpenAI 并没有单一的创始人，而是由多位科技和人工智能领域的重要人物共同成立的。 ChatGPT的爆炸性成功是OpenAI发展史上最戏剧性的篇章，而这一改变世界的产品，在Sam Altman的描述中，"确实是个意外"。 这个意外不仅彻底改变了OpenAI ... 谁有权制定 AI 发展的方向？是少数掌握核心技术的公司，还是更广泛的公众和社会？ 通过保留非营利母体的控制权，OpenAI 试图在这两者之间找到 ...
Thought:```json
{
    "action": "Final Answer",
    "action_input": "OpenAI是一家由多位科技和人工智能领域的重要人物共同创立的组织，其目标是让AI为人类服务。为了确保人工智能对所有人有益，OpenAI致力于研究和开发安全的人工智能技术。他们的研究成果包括著名的GPT系列模型。关键人物包括Sam Altman等，他们在创建过程中发挥了重要作用。"
}
```

> Finished chain.
[回答] OpenAI是一家由多位科技和人工智能领域的重要人物共同创立的组织，其目标是让AI为人类服务。为了确保人工智能对

## 📌思考
- 如果要让智能体拥有“规划一周学习任务”的能力，你会添加哪些工具和模块？

- 搜索引擎工具，获取学习材料。
- 文档处理工具，根据材料生成模板和笔记。
- 日程管理工具，创建和管理日程安排。
- 任务管理工具，创建任务清单，分解每周的学习目标为每日目标。
- 提醒通知，发送提醒通知用户按时完成任务